In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from orbit.models.lgt import LGTMAP, LGTAggregated, LGTFull
from orbit.models.dlt import DLTMAP, DLTAggregated, DLTFull
from orbit.utils.dataset import load_iclaims

from orbit.diagnostics.backtest import grid_search_orbit
from orbit.diagnostics.metrics import smape, mape, wmape

## Load Data

In [2]:
raw_data = load_iclaims()
data = raw_data.copy()

print(data.shape)
data.head(5)

(443, 7)


,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


## LGT Tuning Exampe

In [3]:
lgt = LGTMAP(
    date_col='week',
    response_col='claims',
    seasonality=52,
)

In [4]:
param_grid = {
    'level_sm_input': [0.3, 0.5, 0.8],  
    'seasonality_sm_input': [0.3, 0.5, 0.8],
}

In [5]:
best_params, tuned_df = grid_search_orbit(param_grid, 
                                        model=lgt, 
                                        df=data,
                                        min_train_len=100, incremental_len=100, forecast_len=20, 
                                        metrics=None, criteria=None, verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.3}


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2af9ff5e07f97061bbe4badb0f8a2e36 NOW.
 11%|█         | 1/9 [01:00<08:03, 60.48s/it]

tuning metric:0.0066442
tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.5}


 22%|██▏       | 2/9 [01:04<03:10, 27.20s/it]

...

tuning metric:0.0093521
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.5}


 89%|████████▉ | 8/9 [01:27<00:05,  5.39s/it]

tuning metric:0.0093154
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.8}


100%|██████████| 9/9 [01:31<00:00, 10.16s/it]

tuning metric:0.0079935


In [6]:
tuned_df.head()

,level_sm_input,seasonality_sm_input,metrics
0,0.3,0.3,0.006644
1,0.3,0.5,0.006039
2,0.3,0.8,0.005653
3,0.5,0.3,0.007682
4,0.5,0.5,0.006923


In [7]:
best_params

[{'level_sm_input': 0.3, 'seasonality_sm_input': 0.8}]

## DLT Tuning Example

In [8]:
dlt = DLTMAP(
    date_col='week',
    response_col='claims',
    regressor_col=['trend.unemploy', 'trend.filling', 'trend.job'],
    seasonality=52,
)

In [9]:
param_grid = {
    'regressor_beta_prior': [[0] * 3, [0.05] * 3, [0.1] * 3],  
    'regressor_sigma_prior': [[0.3] * 3, [0.5] * 3, [1] * 3],
}

In [10]:
best_params, tuned_df = grid_search_orbit(param_grid, 
                                        model=dlt, 
                                        df=data,
                                        min_train_len=100, incremental_len=100, forecast_len=20, 
                                        metrics=None, criteria=None, verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'regressor_beta_prior': [0, 0, 0], 'regressor_sigma_prior': [0.3, 0.3, 0.3]}


 11%|█         | 1/9 [00:04<00:33,  4.21s/it]

tuning metric:0.0063926
tuning hyper-params {'regressor_beta_prior': [0, 0, 0], 'regressor_sigma_prior': [0.5, 0.5, 0.5]}


 22%|██▏       | 2/9 [00:08<00:30,  4.35s/it]

...

tuning metric:0.0064326
tuning hyper-params {'regressor_beta_prior': [0.1, 0.1, 0.1], 'regressor_sigma_prior': [0.5, 0.5, 0.5]}


 89%|████████▉ | 8/9 [00:33<00:04,  4.04s/it]

tuning metric:0.0061949
tuning hyper-params {'regressor_beta_prior': [0.1, 0.1, 0.1], 'regressor_sigma_prior': [1, 1, 1]}


100%|██████████| 9/9 [00:37<00:00,  4.11s/it]

tuning metric:0.0062322


In [11]:
tuned_df.head()

,regressor_beta_prior,regressor_sigma_prior,metrics
0,"[0, 0, 0]","[0.3, 0.3, 0.3]",0.006393
1,"[0, 0, 0]","[0.5, 0.5, 0.5]",0.006190
2,"[0, 0, 0]","[1, 1, 1]",0.006617
3,"[0.05, 0.05, 0.05]","[0.3, 0.3, 0.3]",0.006367
4,"[0.05, 0.05, 0.05]","[0.5, 0.5, 0.5]",0.006737


In [12]:
best_params

[{'regressor_beta_prior': [0, 0, 0], 'regressor_sigma_prior': [0.5, 0.5, 0.5]}]